Purpose: Analyze sentiment classifier results for both the Kaggle data and  manually-labeled election tweets.
Author:  Carol Sniegoski
Date:  April 30, 2016
Course: MAS DSE Capstone, Spring 2016

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import numpy as np

#from sklearn.externals import joblib
from sklearn import metrics
from sklearn.metrics import classification

print "done"

done


In [2]:
# Function defs.

# Return string with non-ascii chars removed.
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

# Get class frequencies. Input is a series.
def get_class_frequencies(s):
    counts = s.value_counts()
    total = counts.sum()
    counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )
    return counts.sort_index()

# From newer version of python.
def cohen_kappa_score(y1, y2, labels=None, weights=None):
    confusion = metrics.confusion_matrix(y1, y2, labels=labels)
    n_classes = confusion.shape[0]
    sum0 = np.sum(confusion, axis=0)
    sum1 = np.sum(confusion, axis=1)
    expected = np.outer(sum0, sum1) / np.sum(sum0)

    if weights is None:
        w_mat = np.ones([n_classes, n_classes], dtype=np.int)
        w_mat.flat[:: n_classes + 1] = 0
    elif weights == "linear" or weights == "quadratic":
        w_mat = np.zeros([n_classes, n_classes], dtype=np.int)
        w_mat += np.arange(n_classes)
        if weights == "linear":
            w_mat = np.abs(w_mat - w_mat.T)
        else:
            w_mat = (w_mat - w_mat.T) ** 2
    else:
        raise ValueError("Unknown kappa weighting type.")

    k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
    return 1 - k

print('done')

done


In [3]:
# Locate data dir.
%ls ../data/sentiment/

Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv      debate08_sentiment_tweets.tsv
Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv.bak  debate08_sentiment_tweets.tsv.zip
Labeled_electionTweets_randomSample_allPartitions_04-30-16.csv      hashtag_173_top200_byCandidateAndSentiment_05-03-16.csv
Labeled_electionTweets_randomSample_allPartitions_05-04-16.csv      opinion-lexicon-English.rar
bing-liu_opinion-lexicon-English_d04-13-16.rar


In [4]:
# Define variables.

clean_text = 'ascii_clean'
data_dir = "../data/sentiment/"
datafile_name = "Labeled_electionTweets_randomSample_allPartitions_05-04-16.csv"
datafile = data_dir + datafile_name
print datafile

../data/sentiment/Labeled_electionTweets_randomSample_allPartitions_05-04-16.csv


In [32]:
# Load datafile.
df = pd.read_csv(datafile)
#df=df.rename(columns = {'label':'label_LinearSVC_char3grams_0.75sample'})
print df.shape
df.head(2)

(1859, 16)


,Unnamed: 0,Unnamed: 0.1,text,ascii,ascii_clean,label_LinearSVC_char3grams_0.75sample,label_cas_04-27-16,label_LinearSVC_char3grams_0.75randomsample_2016-04-27,label_RandomForest_char3grams_0.75randomsample_2016-04-28,label_LinearSVC_char3grams_originaldata_2016-04-28,label_RandomForest-d5_char3grams_originaldata_2016-04-28,label_LinearSVC_char3grams_1.8randomsample_2016-04-29,label_cas_subj,label_cas_word-sentiment,label_cas_msg-sentiment,label_cas_msg-sentiment_confidence
0,0,0,"RT @ggreenwald: Clinton sadly never complained about ""overclassification run amok"" when whistleblowers &amp; sources were prosecuted... https:/…","RT @ggreenwald: Clinton sadly never complained about ""overclassification run amok"" when whistleblowers &amp; sources were prosecuted... https:/",rt @ggreenwald clinton sadly never complained about overclassification run amok when whistleblowers sources were prosecuted... url,Negative,Negative,Negative,Negative,Negative,Negative,Negative,1,-1,-1,1
1,1,1,Father: Ted Cruz’s success depends on people of faith https://t.co/sXviGD11Xw https://t.co/T0cdqrNowK,Father: Ted Cruzs success depends on people of faith https://t.co/sXviGD11Xw https://t.co/T0cdqrNowK,father ted cruzs success depends on people of faith url url,Negative,Neutral,Negative,Neutral,Negative,Negative,Neutral,0,0,0,0


In [34]:
# Get label column names.

print df.columns.values

label_col_names = filter( lambda x: x.startswith('label'), df.columns.values )
print label_col_names

['Unnamed: 0' 'Unnamed: 0.1' 'text' 'ascii' 'ascii_clean'
 'label_LinearSVC_char3grams_0.75sample' 'label_cas_04-27-16'
 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27'
 'label_RandomForest_char3grams_0.75randomsample_2016-04-28'
 'label_LinearSVC_char3grams_originaldata_2016-04-28'
 'label_RandomForest-d5_char3grams_originaldata_2016-04-28'
 'label_LinearSVC_char3grams_1.8randomsample_2016-04-29' 'label_cas_subj'
 'label_cas_word-sentiment' 'label_cas_msg-sentiment'
 'label_cas_msg-sentiment_confidence' 'label_cas_word-sentiment_mapped'
 'label_cas_msg-sentiment_mapped']
['label_LinearSVC_char3grams_0.75sample', 'label_cas_04-27-16', 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27', 'label_RandomForest_char3grams_0.75randomsample_2016-04-28', 'label_LinearSVC_char3grams_originaldata_2016-04-28', 'label_RandomForest-d5_char3grams_originaldata_2016-04-28', 'label_LinearSVC_char3grams_1.8randomsample_2016-04-29', 'label_cas_subj', 'label_cas_word-sentiment', 'label_cas_m

In [17]:
print df['label_cas_subj'].value_counts()
print df['label_cas_word-sentiment'].value_counts()
print df['label_cas_msg-sentiment'].value_counts()
print df['label_cas_msg-sentiment_confidence'].value_counts()

1    74
0    21
dtype: int64
-1    34
 1    31
 0    30
dtype: int64
-1    48
 1    30
 0    17
dtype: int64
1    52
0    43
dtype: int64


In [35]:
# Set which column is the gold standard.

truth = 'label_cas_04-27-16'
label_col_names.remove(truth)

print truth
print label_col_names

label_cas_04-27-16
['label_LinearSVC_char3grams_0.75sample', 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27', 'label_RandomForest_char3grams_0.75randomsample_2016-04-28', 'label_LinearSVC_char3grams_originaldata_2016-04-28', 'label_RandomForest-d5_char3grams_originaldata_2016-04-28', 'label_LinearSVC_char3grams_1.8randomsample_2016-04-29', 'label_cas_subj', 'label_cas_word-sentiment', 'label_cas_msg-sentiment', 'label_cas_msg-sentiment_confidence', 'label_cas_word-sentiment_mapped', 'label_cas_msg-sentiment_mapped']


In [69]:
# Reset column names.
print df.columns.values

label_col_names = [ 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27', 
                   'label_RandomForest_char3grams_0.75randomsample_2016-04-28',
                   'label_LinearSVC_char3grams_originaldata_2016-04-28',
                   'label_RandomForest-d5_char3grams_originaldata_2016-04-28',
                   'label_LinearSVC_char3grams_1.8randomsample_2016-04-29' ]
print 'done'

['Unnamed: 0' 'text' 'ascii' 'ascii_clean'
 'label_LinearSVC_char3grams_0.75sample' 'label_cas_04-27-16'
 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27'
 'label_RandomForest_char3grams_0.75randomsample_2016-04-28'
 'label_LinearSVC_char3grams_originaldata_2016-04-28'
 'label_RandomForest-d5_char3grams_originaldata_2016-04-28'
 'label_LinearSVC_char3grams_1.8randomsample_2016-04-29']
done


In [19]:
# Look at class proportions.

for label_col_name in [truth]+label_col_names:
    f = get_class_frequencies(df[label_col_name])
    print label_col_name
    print f
    print
    
print 'done'
    
#f1 = get_class_frequencies(df['label_cas_04-27-16'])
#f2 = get_class_frequencies(df['label'])
#print f1
#print f2

label_cas_04-27-16
Negative    (Count: 98, Frequency: 0.52)
Neutral     (Count: 55, Frequency: 0.29)
Positive    (Count: 37, Frequency: 0.19)
dtype: object

label_LinearSVC_char3grams_0.75sample
Negative    (Count: 671, Frequency: 0.36)
Neutral     (Count: 783, Frequency: 0.42)
Positive    (Count: 405, Frequency: 0.22)
dtype: object

label_LinearSVC_char3grams_0.75randomsample_2016-04-27
Negative    (Count: 671, Frequency: 0.36)
Neutral     (Count: 783, Frequency: 0.42)
Positive    (Count: 405, Frequency: 0.22)
dtype: object

label_RandomForest_char3grams_0.75randomsample_2016-04-28
Negative    (Count: 696, Frequency: 0.37)
Neutral     (Count: 898, Frequency: 0.48)
Positive    (Count: 265, Frequency: 0.14)
dtype: object

label_LinearSVC_char3grams_originaldata_2016-04-28
Negative    (Count: 915, Frequency: 0.49)
Neutral     (Count: 653, Frequency: 0.35)
Positive    (Count: 291, Frequency: 0.16)
dtype: object

label_RandomForest-d5_char3grams_originaldata_2016-04-28
Negative    (Count: 

In [33]:
# Map values to Positive, Negative, Neutral.

#di = {10: "A", 20: "B"}
## The values 10 and 20 are replaced by 'A' and 'B'
#df['col1'].replace(di, inplace=True)
#print(df)

val_map = {1:"Positive", 0:"Neutral", -1:"Negative"}
df['label_cas_word-sentiment_mapped'] = df['label_cas_word-sentiment']
df['label_cas_word-sentiment_mapped'].replace( val_map, inplace=True )
df['label_cas_msg-sentiment_mapped'] = df['label_cas_msg-sentiment']
df['label_cas_msg-sentiment_mapped'].replace( val_map, inplace=True )
print 'done'

print df['label_cas_word-sentiment_mapped'].value_counts()
print df['label_cas_msg-sentiment_mapped'].value_counts()

done
Negative    35
Neutral     32
Positive    31
dtype: int64
Negative    48
Positive    30
Neutral     20
dtype: int64


In [38]:
# Look at metrics for those rows that have both a 'truth' label and a predicted label.

dont_use = ['label_cas_subj', 'label_cas_word-sentiment', 'label_cas_msg-sentiment', 'label_cas_msg-sentiment_confidence']

#y = df[df[truth].notnull()][truth].values
for label_col_name in list( set(label_col_names) - set(dont_use) ):
    
    y = df[ df[truth].notnull() & df[label_col_name].notnull() ][truth].values
    y_pred = df[ df[truth].notnull() & df[label_col_name].notnull() ][label_col_name].values
    #y_pred = df[df[truth].notnull()][label_col_name].values
    
    print label_col_name
    print y.shape 
    print y_pred.shape 
    
    print "accuracy =", metrics.accuracy_score(y, y_pred) 
    #print "cohen's kappa =", cohen_kappa_score(y, y_pred)
    print ' Neg', ' Neu', ' Pos' 
    print metrics.confusion_matrix(y, y_pred) 
    print metrics.classification_report(y, y_pred) 
    print

print 'done'

#y = df[df['label_cas_04-27-16'].notnull()]['label_cas_04-27-16'].values
#y_pred = df[df['label_cas_04-27-16'].notnull()]['label'].values


# Reverse y to make sure it hurts the scores!
#y = y[::-1]




label_LinearSVC_char3grams_0.75sample
(190,)
(190,)
accuracy = 0.5
 Neg  Neu  Pos
[[50 32 16]
 [15 30 10]
 [ 9 13 15]]
             precision    recall  f1-score   support

   Negative       0.68      0.51      0.58        98
    Neutral       0.40      0.55      0.46        55
   Positive       0.37      0.41      0.38        37

avg / total       0.54      0.50      0.51       190


label_LinearSVC_char3grams_originaldata_2016-04-28
(190,)
(190,)
accuracy = 0.415789473684
 Neg  Neu  Pos
[[57 34  7]
 [31 14 10]
 [13 16  8]]
             precision    recall  f1-score   support

   Negative       0.56      0.58      0.57        98
    Neutral       0.22      0.25      0.24        55
   Positive       0.32      0.22      0.26        37

avg / total       0.42      0.42      0.41       190


label_RandomForest-d5_char3grams_originaldata_2016-04-28
(190,)
(190,)
accuracy = 0.515789473684
 Neg  Neu  Pos
[[98  0  0]
 [55  0  0]
 [37  0  0]]
             precision    recall  f1-score   suppor

In [50]:
# Which tagged tweets are positive but don't have positive sentiment words?
test_df = df[ df['label_cas_msg-sentiment_mapped']=="Positive" ]
test_df = test_df[ test_df['label_cas_word-sentiment_mapped']!="Positive"]

print test_df.shape
test_df[['text', 'label_cas_msg-sentiment_mapped', 'label_cas_word-sentiment_mapped']]

(4, 18)


,text,label_cas_msg-sentiment_mapped,label_cas_word-sentiment_mapped
7,RT @jwaxsom: Trump Announces 22 Veterans Group to Receive $6 Million Raised at Iowa Rally https://t.co/4pT5lGYYBB,Positive,Neutral
67,"RT @AnnCoulter: IOWA TRUMP SUPPORTERS! There's no excuse not to caucus. GOP not like Dems: Just show up, register &amp; vote. The end. https:/…",Positive,Negative
93,"RT @shanevanderhart: I've been told there are 2000 ish at the @tedcruz rally in Sioux City, packed house, spilling outside. #iacaucus https…",Positive,Neutral
95,RT @JohnBerman: Waiting for @BernieSanders in Iowa City. https://t.co/7ycFbQWjj9,Positive,Neutral


In [51]:
# Which tagged tweets are negative but don't have negative sentiment words?
test_df = df[ df['label_cas_msg-sentiment_mapped']=="Negative" ]
test_df = test_df[ test_df['label_cas_word-sentiment_mapped']!="Negative"]

print test_df.shape
test_df[['text', 'label_cas_msg-sentiment_mapped', 'label_cas_word-sentiment_mapped']]

(15, 18)


,text,label_cas_msg-sentiment_mapped,label_cas_word-sentiment_mapped
3,RT @LOLGOP: Remarkably apt metaphor for Michigan's GOP. https://t.co/bd5j3Ozltp,Negative,Positive
15,"RT @RealTimers: ""Donald Trump skipped the #GOPDebate - it was like a #Seinfeld episode without Kramer."" - @billmaher #RealTime",Negative,Neutral
17,"It ain't over till it's over Bill Cosby.........,,,,I mean Clinton https://t.co/ygqBxKFFdU",Negative,Neutral
18,US declares 22 Clinton emails ‘top secret’: then-Secretary of State Hillary Rodham Clinton checks her Blackber... https://t.co/9VJJHVCDBj,Negative,Neutral
26,"@keksec__org What do you mean, you have plenty of other socialists choices over there, they are just like Trump.",Negative,Neutral
28,"RT @AdamBaldwin: ""I appreciate his military service, but…"" - @AndreaTantaros #ChickenhawkFallacy \nhttps://t.co/XZKqYutplv",Negative,Positive
34,RT @iowahawkblog: Through the Looking Glass\nhttps://t.co/6AMze4KN9N,Negative,Neutral
35,"@CheriJacobus @Morning_Joe MSNBC used to be the super liberal network, now they're Trump Network News? I wonder what changed?",Negative,Neutral
36,Coconut Oil should be Bernie Sanders running mate,Negative,Neutral
51,"#Politics #News Bernie Sanders tries to be the next Barack Obama in Iowa, not the next Howard... https://t.co/lSfC0ohRN3 #UniteBlue #Tcot",Negative,Neutral


In [121]:
# Locate output dir.
%ls ../data/sentiment

bing-liu_opinion-lexicon-English_d04-13-16.rar  debate08_sentiment_tweets.tsv.zip
debate08_sentiment_tweets.tsv                   opinion-lexicon-English.rar


In [124]:
# Write to csv.

out_dir = "../data/sentiment/"
outfile = out_dir + "Labeled_" + outfile_name
print outfile
df.to_csv(outfile)
print 'done'

../data/sentiment/Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv
done
